# Assignment 6

### Part A

In [160]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [161]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [162]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    exp = 0
    for i in range(0, len(data['Probability'])): 
        exp += ((data['Price in 1 Year'][i] / P) - 1) * data['Probability'][i]
    return exp


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [163]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    var = 0
    for i in range(0, len(data['Probability'])): 
        var += ((((data['Price in 1 Year'][i] / P) - 1) - exp_return(data, P)) ** 2) * data['Probability'][i]
        std = np.sqrt(var)
    return std


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [164]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    exp = exp_return(data,P)
    err = (volatility(data,P) * 1.96) / np.sqrt(3)
    return [exp -  err,exp + err]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [165]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    
    return rf + beta * (market_return - rf)


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [166]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    
    ## Remove all rows whose RET columns render an error when trying to convert them to numeric
    df = df[pd.to_numeric(df['RET'], errors='coerce').notna()]
    
    CUSIP_COUNT = {}
    for i in df.index:
        if df.loc[i,'CUSIP'] in CUSIP_COUNT:
            CUSIP_COUNT[df.loc[i,'CUSIP']] += 1
        else:
            CUSIP_COUNT[df.loc[i,'CUSIP']] = 1
    
    drops = []
    for i in df.index:
        if CUSIP_COUNT[df.loc[i,'CUSIP']] < 500:
            drops.append(True)
        else:
            drops.append(False)
            
    df['drops'] = drops
    df = df[df.drops == False]
    df = df.drop(columns=['drops'])      
    return df


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [167]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    df = data.drop(columns=['PERMNO','date'])
    df.RET = pd.to_numeric(df.RET)
    df = df.groupby('CUSIP')['RET'].mean()
    df = df.to_frame()
    output = pd.DataFrame(data = {
         "CUSIP" : df.index.tolist() ,
          "Average Return" : df.RET.tolist() 
      })
    return output


In [168]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    df = data.drop(columns=['PERMNO','date'])
    df.RET = pd.to_numeric(df.RET)
    df = df.groupby('CUSIP')['RET'].std()
    df = df.to_frame()
    output = pd.DataFrame(data = {
        "CUSIP" : df.index.tolist(),
        "Volatility" : df.RET.tolist()
    })
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [169]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    
    return (ret_data['Average Return']).corr(vol_data.Volatility)